In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler


dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfTest = pd.read_csv('../../../DataSet/trip_test.csv', parse_dates=['start_date', 'end_date'], infer_datetime_format=True)
dfWeather = pd.read_csv('../../../DataSet/weather.csv', parse_dates=['date'], infer_datetime_format=True)
dfStation = pd.read_csv('../../../DataSet/station.csv')

In [2]:
dfTrain = dfTrain[['id', 'duration', 
                   'start_date', 'start_station_id', 
                   'subscription_type']]

dfTest = dfTest[['id', 
                 'start_date', 
                 'start_station_id', 
                 'subscription_type']]

dfTrain['Order'] = pd.Series(xrange(len(dfTrain)))
dfTest['Order'] = pd.Series(xrange(len(dfTest)))

dfWeather = dfWeather[['date', 'zip_code', 'mean_temperature_f', 'events']]

print len(dfTrain)
dfTrain.head()

549961


,id,duration,start_date,start_station_id,subscription_type,Order
0,907649,396,2015-08-27 08:36:00,50,Subscriber,0
1,384043,636,2014-07-28 22:06:00,67,Subscriber,1
2,316176,334,2014-06-09 08:42:00,77,Subscriber,2
3,618874,666,2015-01-26 16:55:00,69,Subscriber,3
4,910977,318,2015-08-29 15:09:00,67,Subscriber,4


In [3]:
dates = {'year':dfTrain.start_date.dt.year,
         'month':dfTrain.start_date.dt.month, 
         'day':dfTrain.start_date.dt.dayofyear, 
         'weekday':dfTrain.start_date.dt.dayofweek, 
         'hour':dfTrain.start_date.dt.hour}

wdates = {'year':dfWeather.date.dt.year, 
          'month':dfWeather.date.dt.month, 
          'day':dfWeather.date.dt.dayofyear}

dfTrain = dfTrain.join(pd.DataFrame(dates))

dates = {'year':dfTest.start_date.dt.year, 
         'month':dfTest.start_date.dt.month, 
         'day':dfTest.start_date.dt.dayofyear, 
         'weekday':dfTest.start_date.dt.dayofweek, 
         'hour':dfTest.start_date.dt.hour}

dfTest = dfTest.join(pd.DataFrame(dates))

subscriptionTypes = dfTrain.subscription_type.unique()
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes
dfTest.subscription_type = dfTest.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

In [4]:
dfWeather = dfWeather.join(pd.DataFrame(wdates))[['year', 'month', 'day', 'zip_code',
                                                  'mean_temperature_f', 'events']]
dfWeather.head()

,year,month,day,zip_code,mean_temperature_f,events
0,2013,8,241,94107,68.0,NaN
1,2013,8,242,94107,69.0,NaN
2,2013,8,243,94107,64.0,NaN
3,2013,9,244,94107,66.0,NaN
4,2013,9,245,94107,69.0,NaN


In [5]:
dfTrain = dfTrain[['Order', 'id', 'duration', 'year', 
                   'month', 'weekday', 'day', 
                   'hour', 'start_station_id', 
                   'subscription_type']]

dfTest = dfTest[['Order', 'id', 'year', 'month', 
                 'weekday', 'day', 
                 'hour', 'start_station_id', 
                 'subscription_type']]

dfTrain.head()

,Order,id,duration,year,month,weekday,day,hour,start_station_id,subscription_type
0,0,907649,396,2015,8,3,239,8,50,0
1,1,384043,636,2014,7,0,209,22,67,0
2,2,316176,334,2014,6,0,160,8,77,0
3,3,618874,666,2015,1,0,26,16,69,0
4,4,910977,318,2015,8,5,241,15,67,0


In [6]:
def cityNameToZipCode(row) :

    if row.city == 'San Francisco' :
        return 94107

    if row.city == 'Redwood City' :
        return 94063

    if row.city == 'Palo Alto' :
        return 94301

    if row.city == 'Mountain View' :
        return 94041

    if row.city == 'San Jose' :
        return 95113


In [7]:
# Preparo el STATION dataframe
dfStation.loc[:,'city'] = dfStation.apply(cityNameToZipCode, axis=1)
dfStation.rename(columns={'id':'start_station_id', 'city':'zip_code'}, inplace=True)
dfStation = dfStation[['start_station_id', 'zip_code']]



In [8]:
dfTrain = dfTrain.merge(dfStation, on='start_station_id')[['Order', 'id', 'duration', 
                                                           'start_station_id', 'subscription_type', 
                                                           'weekday', 'year', 'month', 'day', 
                                                           'hour', 'zip_code']]

dfTest = dfTest.merge(dfStation, on='start_station_id')[['Order', 'id',
                                                           'start_station_id', 'subscription_type', 
                                                           'weekday', 'year', 'month', 'day', 
                                                           'hour', 'zip_code']]

In [9]:
dfTrain = dfTrain.merge(dfWeather, on=['year', 'month', 'day', 'zip_code'])
dfTest = dfTest.merge(dfWeather, on=['year', 'month', 'day', 'zip_code'])

In [10]:
print len(dfTrain)
dfTrain.head()

549961


,Order,id,duration,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f,events
0,0,907649,396,50,0,3,2015,8,239,8,94107,72.0,NaN
1,3700,907916,981,50,0,3,2015,8,239,9,94107,72.0,NaN
2,11801,908620,585,50,0,3,2015,8,239,17,94107,72.0,NaN
3,11876,907429,1772,50,0,3,2015,8,239,7,94107,72.0,NaN
4,12703,907759,510,50,0,3,2015,8,239,8,94107,72.0,NaN


In [11]:
# dfTrain[dfTrain.events.isnull()]

In [12]:
dfTest.loc[dfTest.events.isnull()].head()

,Order,id,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f,events
0,0,504737,60,1,5,2014,10,291,11,94107,65.0,NaN
1,23925,505036,60,0,5,2014,10,291,17,94107,65.0,NaN
2,35015,504958,60,1,5,2014,10,291,15,94107,65.0,NaN
3,44812,505161,60,1,5,2014,10,291,21,94107,65.0,NaN
4,78761,504906,60,1,5,2014,10,291,14,94107,65.0,NaN


In [13]:
def setMeanTemperature(row) :
    if np.isnan(row.mean_temperature_f) :
        
        temps = { 94107:[50.0, 52.5, 54.5, 56.5, 56.5, 59.0, 66.0, 64.5, 62.5, 61.0, 56.5, 50.5], # San Francisco
                  94063:[49.0, 52.5, 55.0, 57.5, 62.0, 66.0, 68.0, 68.5, 67.0, 62.0, 54.5, 49.0], # Redwood City
                  94301:[48.0, 51.5, 54.5, 57.5, 61.5, 65.0, 68.0, 67.0, 66.5, 61.0, 53.5, 48.0], # Palo Alto
                  94041:[51.0, 53.5, 56.0, 59.0, 62.5, 66.5, 68.0, 68.0, 68.0, 63.5, 56.0, 51.0], # Mountain View
                  95113:[51.0, 54.5, 57.0, 59.5, 64.0, 68.5, 71.0, 71.0, 69.5, 64.5, 56.5, 51.5]} # San Jose
        
        return temps[row.zip_code][int(row.month) - 1]
    
    return row.mean_temperature_f

def setMeanWindSpeed(row) :
    if np.isnan(row.mean_wind_speed_mph) :
        
        speed = { 94107:[7.0, 8.0, 10.0, 12.0, 14.0, 14.0, 13.0, 12.0, 11.0, 9.0, 7.0, 7.0], # San Francisco
                  94063:[7.0, 8.0, 9.0, 10.0, 11.0, 11.0, 11.0, 10.0, 9.0, 8.0, 7.0, 7.0],   # Redwood City
                  94301:[6.0, 7.0, 9.0, 10.0, 10.0, 11.0, 10.0, 10.0, 9.0, 7.0, 6.0, 6.0],   # Palo Alto
                  94041:[6.0, 7.0, 9.0, 10.0, 10.0, 11.0, 10.0, 10.0, 9.0, 7.0, 6.0, 6.0],   # Mountain View
                  95113:[5.0, 6.0, 6.0, 7.0, 8.0, 8.0, 7.0, 7.0, 6.0, 5.0, 5.0, 5.0]}        # San Jose
        
        return speed[row.zip_code][int(row.month) - 1]
    
    return row.mean_wind_speed_mph

def fillEvents(row) :
    if pd.isnull(row.events) :
        return 'Clear'
    
    if 'rain' in row.events.lower() :
        return 'Rain'
    
    if 'fog' in row.events.lower() :
        return 'Fog'

# Elimino las rows sin muestreo de temperatura
dfTrain = dfTrain.loc[~dfTrain.mean_temperature_f.isnull()]
dfTrain.head()

,Order,id,duration,start_station_id,subscription_type,weekday,year,month,day,hour,zip_code,mean_temperature_f,events
0,0,907649,396,50,0,3,2015,8,239,8,94107,72.0,NaN
1,3700,907916,981,50,0,3,2015,8,239,9,94107,72.0,NaN
2,11801,908620,585,50,0,3,2015,8,239,17,94107,72.0,NaN
3,11876,907429,1772,50,0,3,2015,8,239,7,94107,72.0,NaN
4,12703,907759,510,50,0,3,2015,8,239,8,94107,72.0,NaN


In [14]:
# Completo los NaN de la tabla de Train
dfTrain.events = dfTrain.apply(fillEvents, axis=1)

# Completo los NaN de la tabla de Test
dfTest.mean_temperature_f = dfTest.apply(setMeanTemperature, axis=1)
dfTest.events = dfTest.apply(fillEvents, axis=1)
# dfTest.mean_wind_speed_mph = dfTest.apply(setMeanWindSpeed, axis=1)

# Cambio los events por enteros
eventTypes = dfTrain.events.unique()
dfTrain.events = dfTrain.events.astype('category', categories=eventTypes).cat.codes
dfTest.events = dfTest.events.astype('category', categories=eventTypes).cat.codes

In [15]:
dfTrain.sort_values(by='Order', inplace=True)
dfTest.sort_values(by='Order', inplace=True)

In [16]:
target = dfTrain.duration
testIDs = dfTest.id
dfTrain.drop(['Order', 'id', 'duration', 'zip_code'], axis=1, inplace=True)
dfTest.drop(['Order', 'id', 'zip_code'], axis=1, inplace=True)

___
### Features utilizadas que demostraron tener mejor resultado con K=707 aprox:

In [17]:
dfTrain.head(12)

,start_station_id,subscription_type,weekday,year,month,day,hour,mean_temperature_f,events
0,50,0,3,2015,8,239,8,72.0,0
245503,67,0,0,2014,7,209,22,67.0,0
192095,77,0,0,2014,6,160,8,68.0,0
220417,69,0,0,2015,1,26,16,51.0,0
433359,67,0,5,2015,8,241,15,71.0,2
276773,65,0,3,2014,10,303,7,63.0,0
355978,64,0,4,2015,8,219,17,69.0,0
463844,71,0,2,2014,10,281,14,64.0,0
175078,62,0,4,2015,8,233,7,66.0,0
90919,76,1,3,2015,4,113,6,58.0,0


In [18]:
scaler = StandardScaler().fit(dfTrain)
train = scaler.transform(dfTrain)
result = pd.DataFrame(train)
result.columns = ['start_station_id', 'subscription_type', 
                  'weekday', 'year', 'month', 'day', 
                  'hour', 'mean_temperature_f', 'events']

In [19]:
result.head(12)

,start_station_id,subscription_type,weekday,year,month,day,hour,mean_temperature_f,events
0,-0.459496,-0.426289,0.339284,1.151253,0.465059,0.576854,-1.118437,1.761761,-0.452720
1,0.535017,-0.426289,-1.375339,-0.311179,0.159865,0.276025,1.956473,0.944036,-0.452720
2,1.120025,-0.426289,-1.375339,-0.311179,-0.145329,-0.215329,-1.118437,1.107581,-0.452720
3,0.652019,-0.426289,-1.375339,1.151253,-1.671298,-1.559031,0.638655,-1.672683,-0.452720
4,0.535017,-0.426289,1.482367,1.151253,0.465059,0.596909,0.419018,1.598216,2.529285
5,0.418016,-0.426289,0.339284,-0.311179,1.075447,1.218623,-1.338073,0.289856,-0.452720
6,0.359515,-0.426289,0.910826,1.151253,0.465059,0.376302,0.858291,1.271126,-0.452720
7,0.769020,-0.426289,-0.232257,-0.311179,1.075447,0.998015,0.199382,0.453401,-0.452720
8,0.242513,-0.426289,0.910826,1.151253,0.465059,0.516688,-1.338073,0.780491,-0.452720
9,1.061524,2.345825,0.339284,1.151253,-0.755716,-0.686627,-1.557709,-0.527868,-0.452720
